In [1]:
!pip -q install langchain huggingface_hub openai google-search-results tiktoken wikipedia


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(usecwd=True))


BASE_URL = os.getenv("LITELLM_BASE_URL", "http://a6k2.dgx:34000/v1")
API_KEY = os.getenv("LITELLM_API_KEY", "")
MODEL_NAME = os.getenv("MODEL_NAME", "qwen3-32b")

In [3]:
!pip show langchain

Name: langchain
Version: 1.0.3
Summary: Building applications with LLMs through composability
Home-page: https://docs.langchain.com/
Author: 
Author-email: 
License: MIT
Location: C:\Users\dinara\Desktop\NLP\venv\Lib\site-packages
Requires: langchain-core, langgraph, pydantic
Required-by: 


## CoT - Chain of thought Reasoning

In [ ]:
from langchain. import OpenAI

ModuleNotFoundError: No module named 'langchain.llms'

In [5]:
from openai import OpenAI

client = OpenAI(
    base_url=BASE_URL,
    api_key=API_KEY
)

# Send "hello" to the agent (model) and capture the response text
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {
            "role": "user", 
            "content": "Explain step by step. How old is the president of the United States?"
        }
    ],
)

result = response.choices[0].message.content
print(result)




As of July 2024, the current President of the United States is **Joe Biden**. Here's a step-by-step breakdown of his age:

1. **Birthdate**: Joe Biden was born on **November 20, 1942**.
2. **Current Year**: The current year is **2024**.
3. **Age Calculation**:
   - Subtract the birth year from the current year: $ 2024 - 1942 = 82 $.
   - Since his birthday (November 20) has **not yet occurred** in 2024 (as of July 2024), he has not yet turned 82.
4. **Result**: As of July 2024, Joe Biden is **81 years old**.

**Note**: If the question is asked after November 20, 2024, his age would be 82. However, based on the context provided (July 2024), the answer remains **81**.


In [6]:
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {
            "role": "user", 
            "content": "How would I get from Singapore to San Francisco?"
        }
    ],
)

result = response.choices[0].message.content
print(result)



To travel from Singapore to San Francisco, the most practical and efficient method is by air. Here's a comprehensive guide to help you plan your journey:

---

### **1. By Air (Recommended)**
#### **Direct Flights**
- **Airlines**: United Airlines operates **non-stop flights** from **Singapore Changi Airport (SIN)** to **San Francisco International Airport (SFO)**. This is the fastest option, taking approximately **16–17 hours**.
  - **Book in advance** for better fares and seat selection.
  - Check for promotions or loyalty programs (e.g., United MileagePlus).

#### **Connecting Flights**
If direct flights are unavailable or more expensive, consider:
- **Emirates**: Singapore → Dubai (DXB) → San Francisco.
- **Cathay Pacific**: Singapore → Hong Kong (HKG) → San Francisco.
- **Qatar Airways**: Singapore → Doha (DOH) → San Francisco.
- **Other options**: Air France (via Paris), British Airways (via London), or Lufthansa (via Frankfurt).
- **Layover duration**: 2–5 hours (depending on 

In [7]:
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {
            "role": "user", 
            "content": "Explain step by step. How would I get from Singapore to San Francisco?"
        }
    ],
)

result = response.choices[0].message.content
print(result)



To travel from Singapore to San Francisco, follow these steps:

---

### **1. Plan Your Flight**
- **Direct Flights**:  
  - **Airlines**: Singapore Airlines (SQ), United Airlines (UA), and Delta Air Lines (DL) offer direct flights from **Changi Airport (SIN)** to **San Francisco International Airport (SFO)**.  
  - **Duration**: ~15–16 hours.  
  - **Booking Tips**: Use platforms like Skyscanner, Google Flights, or airline websites. Book in advance for better prices, especially during off-peak seasons (e.g., avoid holidays or peak travel months).

- **Connecting Flights** (if direct flights are unavailable or expensive):  
  - Popular layover hubs include **Los Angeles (LAX)**, **Dubai (DXB)**, **Narita (NRT)**, or **Tokyo (HND)**.  
  - Total travel time may increase by 4–12 hours depending on layover duration.  
  - Ensure at least 3–4 hours for domestic connections in the U.S. (e.g., from LAX to SFO).

---

### **2. Check Travel Documents**
- **Passport**: Ensure it’s valid for a

## Manual ReACT


In [9]:
question = "How old is the president of the United States?"

manual_react = f"""Question: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?
Thought: I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.
Action: Search[Colorado orogeny]
Observation: The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.
Thought: It does not mention the eastern sector. So I need to look up eastern sector.
Action: Lookup[eastern sector]
Observation: (Result 1 / 1) The eastern sector extends into the High Plains and is called the Central Plains orogeny.
Thought: The eastern sector of Colorado orogeny extends into the High Plains. So I need to search High Plains and find its elevation range.
Action: Search[High Plains]
Observation: High Plains refers to one of two distinct land regions
Thought: I need to instead search High Plains (United States).
Action: Search[High Plains (United States)]
Observation: The High Plains are a subregion of the Great Plains. From east to west, the High Plains rise in elevation from around 1,800 to 7,000 ft (550 to 2,130 m).[3]
Thought: High Plains rise in elevation from around 1,800 to 7,000 ft, so the answer is 1,800 to 7,000 ft.
Action: Finish[1,800 to 7,000 ft]

Question: Musician and satirist Allie Goertz wrote a song about the "The Simpsons" character Milhouse, who Matt Groening named after who?
Thought: The question simplifies to "The Simpsons" character Milhouse is named after who. I only need to search Milhouse and find who it is named after.
Action: Search[Milhouse]
Observation: Milhouse Mussolini Van Houten is a recurring character in the Fox animated television series The Simpsons voiced by Pamela Hayden and created by Matt Groening.
Thought: The paragraph does not tell who Milhouse is named after, maybe I can look up "named after".
Action: Lookup[named after]
Observation: (Result 1 / 1) Milhouse was named after U.S. president Richard Nixon, whose middle name was Milhous.
Thought: Milhouse was named after U.S. president Richard Nixon, so the answer is Richard Nixon.
Action: Finish[Richard Nixon]

Question: Which documentary is about Finnish rock groups, Adam Clayton Powell or The Saimaa Gesture?
Thought: I need to search Adam Clayton Powell and The Saimaa Gesture, and find which documentary is about Finnish rock groups.
Action: Search[Adam Clayton Powell]
Observation: Could not find [Adam Clayton Powell]. Similar: [’Adam Clayton Powell III’, ’Seventh Avenue (Manhattan)’, ’Adam Clayton Powell Jr. State Office Building’, ’Isabel Washington Powell’, ’Adam Powell’, ’Adam Clayton Powell (film)’, ’Giancarlo Esposito’].
Thought: To find the documentary, I can search Adam Clayton Powell (film).
Action: Search[Adam Clayton Powell (film)]
Observation: Adam Clayton Powell is a 1989 American documentary film directed by Richard Kilberg. The film is about the rise and fall of influential African-American politician Adam Clayton Powell Jr.[3][4] It was later aired as part of the PBS series The American Experience.
Thought: Adam Clayton Powell (film) is a documentary about an African-American politician, not Finnish rock groups. So the documentary about Finnish rock groups must instead be The Saimaa Gesture.
Action: Finish[The Saimaa Gesture]

Question: What profession does Nicholas Ray and Elia Kazan have in common?
Thought: I need to search Nicholas Ray and Elia Kazan, find their professions, then find the profession they have in common.
Action: Search[Nicholas Ray]
Observation: Nicholas Ray (born Raymond Nicholas Kienzle Jr., August 7, 1911 - June 16, 1979) was an American film director, screenwriter, and actor best known for the 1955 film Rebel Without a Cause.
Thought: Professions of Nicholas Ray are director, screenwriter, and actor. I need to search Elia Kazan next and find his professions.
Action: Search[Elia Kazan]
Observation: Elia Kazan was an American film and theatre director, producer, screenwriter and actor.
Thought: Professions of Elia Kazan are director, producer, screenwriter, and actor. So profession Nicholas Ray and Elia Kazan have in common is director, screenwriter, and actor.
Action: Finish[director, screenwriter, actor]

Question: Which magazine was started first Arthur’s Magazine or First for Women?
Thought: I need to search Arthur’s Magazine and First for Women, and find which was started first.
Action: Search[Arthur’s Magazine]
Observation: Arthur’s Magazine (1844-1846) was an American literary periodical published in Philadelphia in the 19th century.
Thought: Arthur’s Magazine was started in 1844. I need to search First for Women next.
Action: Search[First for Women]
Observation: First for Women is a woman’s magazine published by Bauer Media Group in the USA.[1] The magazine was started in 1989.
Thought: First for Women was started in 1989. 1844 (Arthur’s Magazine) < 1989 (First for Women), so Arthur’s Magazine was started first.
Action: Finish[Arthur’s Magazine]

Question:{question}"""

In [10]:
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {
            "role": "user", 
            "content": manual_react
        }
    ],
)

result = response.choices[0].message.content
print(result)



The current president of the United States is Joe Biden, who was born on November 20, 1942. As of October 31, 2023, he is **80 years old**, as his birthday in November 2023 has not yet occurred. After November 20, 2023, he will turn 81. 

**Answer:** 80 years old (as of October 31, 2023).


## ReACT

“REACT: SYNERGIZING REASONING AND ACTING IN LANGUAGE MODELS” (Shunyu et al., 2022)

In [ ]:
from langchain import OpenAI, Wikipedia
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.agents.react.base import DocstoreExplorer

In [11]:

docstore=DocstoreExplorer(Wikipedia())

tools = [
    Tool(
        name="Search",
        func=docstore.search,
        description="useful for when you need to ask with search"
    ),
    Tool(
        name="Lookup",
        func=docstore.lookup,
        description="useful for when you need to ask with lookup"
    )
]

llm =  OpenAI(
    base_url=BASE_URL,
    api_key=API_KEY,
)


react = initialize_agent(tools, llm, agent=AgentType.REACT_DOCSTORE, verbose=True)

NameError: name 'DocstoreExplorer' is not defined

In [ ]:
question = "How old is the president of the United States?"
react.run(question)



> Entering new AgentExecutor chain...

Thought: I need to search the president of the United States, find his age, and then finish.

Action: Search[president of the United States]

Observation: The President of the United States (POTUS) is the head of state and head of government of the United States of America. The president directs the executive branch of the federal government and is the commander-in-chief of the United States Armed Forces.
The power of the presidency has grown substantially since the first president, George Washington, took office in 1789. While presidential power has ebbed and flowed over time, the presidency has played an increasingly significant role in American political life since the beginning of the 20th century, with notable expansion during the presidency of Franklin D. Roosevelt. In modern times, the president is one of the world's most powerful political figures – the leader of the only remaining global superpower. As the leader of the nation with the 

'78 years old'

In [ ]:
question = "How old will the president of the United States be in June 2023?"
react.run(question)



> Entering new AgentExecutor chain...

Thought: I need to search the president of the United States, find their age, then find how old they will be in June 2023.

Action: Search[president of the United States]

Observation: The President of the United States (POTUS) is the head of state and head of government of the United States of America. The president directs the executive branch of the federal government and is the commander-in-chief of the United States Armed Forces.
The power of the presidency has grown substantially since the first president, George Washington, took office in 1789. While presidential power has ebbed and flowed over time, the presidency has played an increasingly significant role in American political life since the beginning of the 20th century, with notable expansion during the presidency of Franklin D. Roosevelt. In modern times, the president is one of the world's most powerful political figures – the leader of the only remaining global superpower. As the 

'80 years old'

In [ ]:
question = "How old is the president of the United States?"
react.run(question)

#### Let pull apart the Chain

In [ ]:
print(react.agent.llm_chain.prompt.template)
the



Question: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?
Thought: I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.
Action: Search[Colorado orogeny]
Observation: The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.
Thought: It does not mention the eastern sector. So I need to look up eastern sector.
Action: Lookup[eastern sector]
Observation: (Result 1 / 1) The eastern sector extends into the High Plains and is called the Central Plains orogeny.
Thought: The eastern sector of Colorado orogeny extends into the High Plains. So I need to search High Plains and find its elevation range.
Action: Search[High Plains]
Observation: High Plains refers to one of two distinct land regions
Thought: I need to instead search High Plains (United States).
Action: Search[High Plains (United St

In [ ]:
# multi hop example
question = "What is the first film that Russell Crowe won an Oscar for, and who directed that movie?"
react.run(question)



> Entering new AgentExecutor chain...
Thought: I need to search Russell Crowe and find the first film he won an Oscar for, and who directed that movie.
Action: Search[Russell Crowe]
Observation: Could not find [Russell Crowe]. Similar: ['Russell Crowe', 'Russell Crowe filmography', "The Pope's Exorcist", 'Unhinged (2020 film)', 'Robin Hood (2010 film)', "Russell Crowe's jockstrap", 'Dave Crowe', 'Sleeping Dogs (upcoming film)', 'List of awards and nominations received by Russell Crowe', 'Gladiator (2000 film)']
Thought: To find the first film Russell Crowe won an Oscar for, I can search Gladiator (2000 film).
Action: Search[Gladiator (2000 film)]
Observation: Gladiator is a 2000 epic historical drama film directed by Ridley Scott and written by David Franzoni, John Logan, and William Nicholson. It was produced by Universal Pictures. DreamWorks Pictures distributed the film in North America while Universal Pictures released it internationally through United International Pictures. It 

'Gladiator, Ridley Scott'

In [ ]:
import langchain

langchain.debug = True

In [ ]:
# multi hop example
question = "What is the first film that Russell Crowe won an Oscar for, and who directed that movie?"
react.run(question)

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "What is the first film that Russell Crowe won an Oscar for, and who directed that movie?"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "What is the first film that Russell Crowe won an Oscar for, and who directed that movie?",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:OpenAI] Entering LLM run with input:
{
  "prompts": [
    "Question: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?\nThought: I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.\nAction: Search[Colorado orogeny]\nObservation: The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.\nThought: It doe

'Gladiator (2000 film), Ridley Scott'

In [ ]:
langchain.debug = False